# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [27]:
# checking your current working directory
print("Current working directory:", os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_paths = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    file_paths.append(file_path_list)
    
# Remove checkpoint csv file
file_paths = file_paths[0]
file_paths = [x for x in file_paths if 'checkpoint' not in x]
print(file_paths)

Current working directory: /home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/eve

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [28]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_paths:

    # Print files processed
    print(f)
    
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
         # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
    
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the 
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    # Write all the rows in the seperate csvs into the event_datafile_new.csv
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


/home/workspace/event_data/2018-11-27-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/workspace/event_data/2018-11-07-events.csv
/home/workspace/event_data/2018-11-09-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-24-events.csv
/home/workspace/event_data/2018-11-29-events.csv
/home/workspace/event_data/2018-11-15-events.csv
/home/workspace/event_data/2018-11-20-events.csv
/home/workspace/event_data/2018-11-06-events.csv
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-21-events.csv
/home/workspace/event_data/2018-11-10-events.csv
/home/workspace/event_data/2018-11-23-events.csv
/home/workspace/event_data/2018-11-02-events.csv
/home/workspace/event_data/2018-11-28-events.csv
/home/workspace/even

In [29]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster & connect to Database

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we will need to create tables to run the following queries. Recall that with Apache Cassandra, you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## --------------------------------------------------------------------------------------------------------------------------------------------------


# 1. Requirements: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
We will model the table after the following query: 
`SELECT artist, song, length FROM song_session WHERE session_id = 338
    AND item_in_session = 4`


In [16]:
## Declare CREATE and DROP statements
create_table_1_query = """
    CREATE TABLE IF NOT EXISTS song_session (
        session_id int, item_in_session int, 
        artist text, 
        length float, song text,
        PRIMARY KEY (session_id, item_in_session)
        )
"""
# NTS: Cassandra does not have VARCHAR(2)

drop_table_1_query = """
    DROP TABLE IF EXISTS song_session
"""

In [31]:
# Create table
try:
    session.execute(create_table_1_query)
except Exception as e:
    print(e)

In [32]:
# INSERT data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = """
            INSERT INTO song_session (
                session_id, item_in_session, 
                artist, length, song
                )
            VALUES (%s, %s, %s, %s, %s)
        """
        
        try: 
            session.execute(query, ( int(line[8]), int(line[3]), line[0], float(line[5]), line[9]) )
        except Exception as e:
            print(e)
            break

#### Query Description: In this query, I used session_id and item_in_session as the composite primary key. Each row is uniquely identified by a combination of session_id + item_in_session.

In [33]:
select_query_1 = """
    SELECT artist, song, length FROM song_session WHERE session_id = 338
    AND item_in_session = 4
"""    

try:
    rows = session.execute(select_query_1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


# 2. Requirements: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
We will model the table with the following query: 
`SELECT artist, song, first_name, last_name FROM song_playlist_session WHERE session_id = 182
    AND user_id = 10`

In [34]:
# Declare CREATE and DROP statements
create_table_2_query = """
    CREATE TABLE IF NOT EXISTS song_playlist_session (
        session_id int,  user_id int, item_in_session int, 
        artist text, first_name varchar, last_name varchar, 
        song text,
        PRIMARY KEY ((session_id, user_id), item_in_session)
        )
    WITH CLUSTERING ORDER BY (item_in_session ASC)
"""

drop_table_2_query = """
    DROP TABLE IF EXISTS song_playlist_session
"""              

In [35]:
# CREATE TABLE
try:
    session.execute(create_table_2_query)
except Exception as e:
    print(e)

In [36]:
# INSERT data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = """
            INSERT INTO song_playlist_session (
                session_id,  user_id, item_in_session, 
                artist, first_name, last_name, 
                song
                )
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        
        try:
            session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[1], line[4], line[9]) )
        except Exception as e:
            print(e)
            break

#### Query Description: In this query, I used session_id and item_in_session as the partition keys, as these 2 can uniquely identify each row. item_in_session is used as a clustering column. Within each partition, they are sorted by item_in_session

In [38]:
select_query_2 = """
    SELECT artist, song, first_name, last_name FROM song_playlist_session WHERE session_id = 182
    AND user_id = 10
"""    

try:
    rows = session.execute(select_query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


# 3.Requirements: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
We will model the table after the query: 
`SELECT first_name, last_name FROM user_song WHERE song = 'All Hands Against His Own'`

`PRIMARY KEY is declared ((song), user_id)`. song is the partition key, user_id is also used in the primary key and is the clustering column

user_id is used as it can be used to uniquely identify each row. 

In [40]:
## Declare CREATE and DROP statements
create_table_3_query = """
    CREATE TABLE IF NOT EXISTS user_song (
        song text, user_id int, 
        first_name varchar, last_name varchar, 
        PRIMARY KEY ((song), user_id)
        )
"""

drop_table_3_query = """
    DROP TABLE IF EXISTS user_song
"""                              

In [41]:
# CREATE TABLE
try:
    session.execute(create_table_3_query)
except Exception as e:
    print(e)

In [44]:
# INSERT data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = """
            INSERT INTO user_song (
                song, user_id,
                first_name, last_name
                )
            VALUES (%s, %s, %s, %s)
        """
        
        try: 
            session.execute(query, (line[9], int(line[10]), line[1], line[4]) )                     
        except Exception as e:
            print(e)
            break

#### Query Description: In this query, I used song and user_id as the primary keys, as these 2 can uniquely identify each row. song is used as the PARTITION KEY while user_id is used as the CLUSTERING COLUMN

In [45]:
select_query_3 = """
    SELECT first_name, last_name FROM user_song WHERE song = 'All Hands Against His Own'
"""    

try:
    rows = session.execute(select_query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [46]:
try:
    session.execute(drop_table_1_query)
    session.execute(drop_table_2_query)
    session.execute(drop_table_3_query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [47]:
session.shutdown()
cluster.shutdown()